In [3]:
# ======================================================
# STEP 5-B: CBM Model Evaluation
# ======================================================
# - 타깃: 이전직장사항_전직유무 (이직 성공)
# - 목적: 개인 이직 성공 예측 + precision 중심 검증
# - 정책 타깃(진성쉬었음) 사용 ❌
# ======================================================

import numpy as np
import pandas as pd

from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score,
    classification_report
)
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------------------
# 데이터 & 모델 로드
# ------------------------------------------------------
print("=" * 70)
print("🚀 STEP 5-B: CBM MODEL EVALUATION START")
print("=" * 70)

X_TEST_PATH = "../X_test.csv"
Y_TEST_PATH = "../y_test.csv"
MODEL_PATH  = "../models/RYUI/cat_model.cbm"

X_test = pd.read_csv(X_TEST_PATH)
y_test = pd.read_csv(Y_TEST_PATH)

y_true = (
    y_test.iloc[:, 0]
    .astype(int)
    .to_numpy()
)

cat = CatBoostClassifier()
cat.load_model(MODEL_PATH)

# ------------------------------------------------------
# 예측
# ------------------------------------------------------
y_prob = cat.predict_proba(X_test)[:, 1]

# ------------------------------------------------------
# 기본 threshold (0.5) — 참고용
# ------------------------------------------------------
thr = 0.5
y_pred = (y_prob >= thr).astype(int)

print("\n📌 기본 threshold = 0.5")
print(f"Accuracy  : {accuracy_score(y_true, y_pred):.4f}")
print(f"Precision : {precision_score(y_true, y_pred, zero_division=0):.4f}")
print(f"Recall    : {recall_score(y_true, y_pred, zero_division=0):.4f}")
print(f"F1-score  : {f1_score(y_true, y_pred, zero_division=0):.4f}")
print(f"AUC       : {roc_auc_score(y_true, y_prob):.4f}")

# ------------------------------------------------------
# Precision 우선 threshold 탐색
# ------------------------------------------------------
print("\n📌 Precision 중심 threshold 탐색")

thresholds = np.arange(0.05, 0.96, 0.01)
best = None

for t in thresholds:
    pred = (y_prob >= t).astype(int)
    p = precision_score(y_true, pred, zero_division=0)
    r = recall_score(y_true, pred, zero_division=0)
    f = f1_score(y_true, pred, zero_division=0)

    if p >= 0.7:
        if best is None or f > best["f1"]:
            best = {"thr": t, "precision": p, "recall": r, "f1": f}

if best:
    print("Best threshold:", round(best["thr"], 2))
    print("Precision     :", round(best["precision"], 4))
    print("Recall        :", round(best["recall"], 4))
    print("F1            :", round(best["f1"], 4))
else:
    print("⚠️ Precision ≥ 0.7 만족 threshold 없음")

# ------------------------------------------------------
# 리포트
# ------------------------------------------------------
print("\n📊 Classification Report (threshold=0.5)")
print(classification_report(y_true, y_pred, digits=4))

print("=" * 70)
print("🎉 STEP 5-B COMPLETE")
print("=" * 70)


🚀 STEP 5-B: CBM MODEL EVALUATION START

📌 기본 threshold = 0.5
Accuracy  : 0.8214
Precision : 0.7563
Recall    : 0.3240
F1-score  : 0.4536
AUC       : 0.8297

📌 Precision 중심 threshold 탐색
Best threshold: 0.42
Precision     : 0.7035
Recall        : 0.379
F1            : 0.4926

📊 Classification Report (threshold=0.5)
              precision    recall  f1-score   support

           0     0.8285    0.9690    0.8933      9009
           1     0.7563    0.3240    0.4536      2673

    accuracy                         0.8214     11682
   macro avg     0.7924    0.6465    0.6735     11682
weighted avg     0.8120    0.8214    0.7927     11682

🎉 STEP 5-B COMPLETE
